## Importando bibliotecas necessárias

In [1]:
import fitz
import pandas as pd
from PIL import Image
import pytesseract
import io
import re


## Carregamento e processamento do PDF

In [2]:

pdf_path = 'C:\\Users\\BlueShift\\Downloads\\apostila-python.pdf'
doc = fitz.open(pdf_path)

path_tesseract = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
pytesseract.Tesseract_cmd = path_tesseract

## Removendo as propagandas

In [3]:
def remove_ad_alura(text):
    ad_text = """Agora é a melhor hora de aprender algo novo
Se você está gostando dessa apostila, certamente vai aproveitar os cursos
online que lançamos na plataforma Alura. Você estuda a qualquer momento
com a qualidade Caelum. Programação, Mobile, Design, Infra, Front-End e
Business, entre outros! Ex-estudante da Caelum tem 10% de desconto, siga o link!
Conheça a Alura Cursos Online."""
    return text.replace(ad_text, '')
 
def remove_ad_casa_do_codigo(text):
    ad_text = """Seus livros de tecnologia parecem do século passado?
Conheça a Casa do Código, uma nova editora, com autores de destaque no
mercado, foco em ebooks (PDF, epub, mobi), preços imbatíveis e assuntos
atuais.
Com a curadoria da Caelum e excelentes autores, é uma abordagem diferente
para livros de tecnologia no Brasil.
Casa do Código, Livros de Tecnologia."""
    return text.replace(ad_text, '')
 
def remove_ad_caelum(text):
    ad_text = """Você pode também fazer o curso data dessa apostila na Caelum
Querendo aprender ainda mais sobre? Esclarecer dúvidas dos exercícios? Ouvir
explicações detalhadas com um instrutor?
A Caelum oferece o curso data presencial nas cidades de São Paulo, Rio de
Janeiro e Brasília, além de turmas incompany.
Consulte as vantagens do curso Python e Orientação a Objetos"""
    return text.replace(ad_text, '')

## Extraindo texto e informações das imagens

In [4]:
def extract_text_from_image(page):
    image_list = page.get_images(full=True)
    image_text = ""
    for image_index in image_list:
        base_image = doc.extract_image(image_index[0])
        image_bytes = base_image["image"]

        image = Image.open(io.BytesIO(image_bytes))

        image_text += pytesseract.image_to_string(image) + "\n"

    return image_text

def extract_info(df):
    data = {'Capitulo': [], 'Seção': [], 'Contexto': []}

    current_chapter = ""
    current_section = ""
    content_buffer = ""
    capture_content_immediately = True

    for _, row in df.iterrows():
        text = row['Text']
        
        matches = re.finditer(rf"({charpter}|{section})\s*((?:.(?!{charpter}|{section}))*.?)", text, re.DOTALL)

        for match in matches:
            header, content = match.groups()
            header = header.strip().replace('\n', ' ')

            if 'CAPÍTULO' in header:
                if content_buffer:
                    data['Capitulo'].append(current_chapter)
                    data['Seção'].append(current_section)
                    data['Contexto'].append(content_buffer.strip())
                    content_buffer = ""
                current_chapter = header
                current_section = ""
                capture_content_immediately = True
            else:
                if header != current_section or capture_content_immediately:
                    if content_buffer:
                        data['Capitulo'].append(current_chapter)
                        data['Seção'].append(current_section)
                        data['Contexto'].append(content_buffer.strip())
                        content_buffer = ""
                    current_section = header
                    capture_content_immediately = False
                content_buffer += " " + content.strip()

    if content_buffer:
        data['Capitulo'].append(current_chapter)
        data['Seção'].append(current_section)
        data['Contexto'].append(content_buffer.strip())

    return data

data = {'Text': []}

start_page = 6
image_pages = [15, 45, 47, 46, 47, 131, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 204, 214, 215, 216, 217]

for page_num in range(start_page, doc.page_count):
    page = doc.load_page(page_num)
    page_text = page.get_text("text", sort=True)

    page_text = remove_ad_alura(page_text)
    page_text = remove_ad_casa_do_codigo(page_text)
    page_text = remove_ad_caelum(page_text)

    if page_num in image_pages:
        page_text += extract_text_from_image(page)

    data['Text'].append(page_text)

df = pd.DataFrame(data)

charpter = r"CAPÍTULO \d+\n[^\n]+"
section = r"(?<!\d)\d{1,2}\.\d (?![\d-])[A-Za-zÀ-ÖØ-öø-ÿ][^\n]+"

extracted_structure = extract_info(df, doc)
extracted_dataframe = pd.DataFrame(extracted_structure)

doc.close()

In [5]:
display(extracted_dataframe) 

,Capitulo,Seção,Contexto
0,CAPÍTULO 1 COMO APRENDER PYTHON,1.2 SOBRE OS EXERCÍCIOS,1 COMO APRENDER PYTHON \n1
1,CAPÍTULO 1 COMO APRENDER PYTHON,1.3 TIRANDO DÚVIDAS E INDO ALÉM,"Para tirar dúvidas de exercícios, ou de Python..."
2,CAPÍTULO 2 O QUE É PYTHON,2.2 BREVE HISTÓRIA,Python foi criada em 1990 por Guido Van Rossum...
3,CAPÍTULO 2 O QUE É PYTHON,2.3 INTERPRETADOR,Você provavelmente já ouviu ou leu em algum lu...
4,CAPÍTULO 2 O QUE É PYTHON,2.4 QUAL VERSÃO UTILIZAR?,"Para quem está começando, a primeira dúvida na..."
...,...,...,...
115,CAPÍTULO 16 APÊNDICE - INSTALAÇÃO,16.2 INSTALANDO O PYTHON NO LINUX,Os sistemas operacionais baseados no Debian já...
116,CAPÍTULO 16 APÊNDICE - INSTALAÇÃO,16.3 INSTALANDO O PYTHON NO MACOS,A maneira mais fácil de instalar o Python3 no ...
117,CAPÍTULO 16 APÊNDICE - INSTALAÇÃO,16.4 OUTRAS FORMAS DE UTILIZAR O PYTHON,Podemos rodar o Python diretamente do seu próp...
118,CAPÍTULO 16 APÊNDICE - INSTALAÇÃO,16.2 INSTALANDO O PYTHON NO LINUX,213


## Salvando como arquivo .csv

In [8]:
extracted_dataframe.to_csv('dataframe_apostila.csv', index=False)
print("Dados salvo com sucesso")

Dados salvo com sucesso
